In [1]:
!pip install einops
!pip install transformers[torch] -U
!pip install accelerate -U

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
import torch

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
cd '/content/drive/MyDrive/Notebooks'

/content/drive/.shortcut-targets-by-id/1-znWgzrFEMtD1TIZZNzZVN2c_-pbU6To/Notebooks


In [6]:
# load model
checkpoint = "Salesforce/codegen-350M-mono"
model = AutoModelForCausalLM.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [10]:
import os

train_texts = []
directory_path = "code/code"

for filename in os.listdir(directory_path):
  file_path = os.path.join(directory_path, filename)
  if os.path.isfile(file_path):
      with open(file_path, 'r') as file:
          content = file.read()
          train_texts.append(content)

In [11]:
print(tokenizer.model_max_length)

2048


In [12]:
#max_token_limit = tokenizer.model_max_length
max_token_limit = 512
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
encoded_inputs = tokenizer(train_texts, return_tensors="pt", padding=True, truncation=True, max_length=max_token_limit)

In [13]:
training_dataset = {
    "input_ids": encoded_inputs["input_ids"],
    "attention_mask": encoded_inputs["attention_mask"],
    "labels": encoded_inputs["input_ids"],  # Use the same input_ids as labels for language modeling
}

In [ ]:
print(training_dataset)

{'input_ids': tensor([[  198,  9487,  1391,  ..., 50295, 50295, 50295],
        [ 9487,  1391,   198,  ..., 50295, 50295, 50295],
        [11627,  3004,  1391,  ..., 50295, 50295, 50295],
        ...,
        [  198,  9487,  1391,  ..., 50295, 50295, 50295],
        [  198,  9487,  1391,  ..., 50295, 50295, 50295],
        [  198,  9487,  1391,  ..., 50295, 50295, 50295]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([[  198,  9487,  1391,  ..., 50295, 50295, 50295],
        [ 9487,  1391,   198,  ..., 50295, 50295, 50295],
        [11627,  3004,  1391,  ..., 50295, 50295, 50295],
        ...,
        [  198,  9487,  1391,  ..., 50295, 50295, 50295],
        [  198,  9487,  1391,  ..., 50295, 50295, 50295],
        [  198,  9487,  1391,  ..., 50295, 50295, 50295]])}


In [7]:
from torch.utils.data import Dataset, DataLoader

In [14]:
class CustomDataset(Dataset):
    def __init__(self, text_list, tokenizer, max_token_limit):
        self.tokenizer = tokenizer
        self.text_list = text_list
        self.max_token_limit = max_token_limit

    def __len__(self):
        return len(self.text_list)

    def __getitem__(self, idx):
        text = self.text_list[idx]
        encoded_input = self.tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=self.max_token_limit)
        return {
            "input_ids": encoded_input["input_ids"].squeeze(),
            "attention_mask": encoded_input["attention_mask"].squeeze(),
            "labels": encoded_input["input_ids"].squeeze(),
        }

In [15]:
custom_dataset = CustomDataset(train_texts, tokenizer, max_token_limit)

In [16]:
training_args = TrainingArguments(
    output_dir="./output",           # Output directory
    num_train_epochs=2,              # Number of training epochs
    per_device_train_batch_size=4,   # Batch size per GPU/CPU
    save_steps=100,               # Save checkpoint every 10000 steps
    save_total_limit=20,              # Limit the total number of saved checkpoints
    logging_dir="./logs",            # Directory for storing logs
    logging_steps=100,               # Log training progress every 100 steps
    overwrite_output_dir=True,       # Overwrite the content of the output directory
    save_strategy='steps'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=custom_dataset
)

In [18]:
torch.cuda.empty_cache()

In [19]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


KeyboardInterrupt: ignored

In [21]:
model_1300 = AutoModelForCausalLM.from_pretrained("output/checkpoint-1200")


OSError: ignored

In [ ]:
model_1300.eval()

CodeGenForCausalLM(
  (transformer): CodeGenModel(
    (wte): Embedding(51200, 1024)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-19): 20 x CodeGenBlock(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): CodeGenAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (qkv_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (out_proj): Linear(in_features=1024, out_features=1024, bias=False)
        )
        (mlp): CodeGenMLP(
          (fc_in): Linear(in_features=1024, out_features=4096, bias=True)
          (fc_out): Linear(in_features=4096, out_features=1024, bias=True)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=51200, bias=True)
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Salesforce/codegen-350M-mono")

In [ ]:
text = """Class {
	#name : #EFInternalTest,"""

In [ ]:
max_length = 512
max_new_tokens = 128

In [ ]:
completion = model_1300.generate(**tokenizer(text, return_tensors="pt"), max_length=max_length, max_new_tokens=max_new_tokens)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=100) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


In [ ]:
print(tokenizer.decode(completion[0]))

Class {
	#name : #EFInternalTest,
	#superclass : #EFAbstractTest,
	#category : #'EnlumineurFormatter-Tests'
}

{ #category : #tests }
EFInternalTest >> testBasicFormat [

	| result |
	result := EFExternalFormat
		          source: '{1}'
		          format: {1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1
